In [55]:
import pandas as pd
import numpy as np
import re

In [56]:
def convert_money(value):
    value = value.replace(",", " ")
    money = int("".join(value.split(" ")[:-1]))
    if value.split(" ")[-1] == "RON":
        money = round(money / 5)
    return money

In [57]:
def convert_area(value):
    value = value.replace(",", ".")
    area = float("".join(value.split(" ")[0]))
    return area

In [58]:
def number_of_floors(value):
    new_value = None
    if value in [np.nan]:
        new_value = np.nan
    elif "/" in value:
        new_value = value.split("/")[1]
    else:
        new_value = np.nan
    return new_value

In [59]:
def apartment_floor(value):
    new_value = None
    if value in [np.nan]:
        new_value = value
    else:
        new_value = re.sub('[<>!,.?*]', "", value.split("/")[0])
        new_value = new_value.strip()
    return new_value

In [60]:
def get_year(value, text):
    new_value = value
    if value in [np.nan]:
        new_values = re.findall("[0-2][0-9][0-9][0-9]", text)
        if new_values == []:
            new_value = np.nan
        else:
            for val in new_values:
                if int(val) < 1900:
                    continue
                else:
                    new_value = val
                    break
    return new_value
    

In [61]:
def parcare_or_garaj(value, text):
    new_value = value
    if text not in [np.nan]:
        text = text.lower()
        words_list = ["garaj", "parcare"]
        if value in [np.nan]:
            for word in words_list:
                if word in text:
                    new_value = word
                    break
    return new_value
            

In [62]:
def building_floors(value, text):
    new_value = value
    flag = 0
    forms = ["\s[0-9][0-9]/[0-9][0-9]\s", "\s[0-9]/[0-9][0-9]\s", "\s[0-9]/[0-9]\s", "parter/[0-9][0-9]", "parter/[0-9]", "demisol/[0-9][0-9]",
             "demisol/[0-9]", "mansardă/[0-9][0-9]", "mansardă/[0-9]", "mansarda/[0-9][0-9]", "mansarda/[0-9]"]
    forms_2 = ['[0-9][0-9] etaje', '[0-9] etaje', '[0-9][0-9] nivele', '[0-9] nivele']
    forms_3 = ['etajul\s[0-9][0-9]/[0-9][0-9]', 'etajul\s[0-9]/[0-9][0-9]', "etajul\s[0-9]/[0-9]", 'etajul\s[0-9][0-9]\sdin\s[0-9][0-9]',
               'etajul\s[0-9]\sdin\s[0-9][0-9]', "etajul\s[0-9]\sdin\s[0-9]", "etaj\s[0-9][0-9]/[0-9][0-9]", "etaj\s[0-9]/[0-9][0-9]",
               "etaj\s[0-9]/[0-9]", 'etaj\s[0-9][0-9]\sdin\s[0-9][0-9]', 'etaj\s[0-9]\sdin\s[0-9][0-9]', "etaj\s[0-9]\sdin\s[0-9]"]
    forms_4 = ['etajul\s[0-9][0-9]', 'etajul\s[0-9]']
    if text not in [np.nan]:
        text = text.lower()
    if (value in [np.nan]) or ("/" not in value):
        flag = 0
        for form in forms:
            new_values = re.findall(form, text)
            if len(new_values) > 0:
                new_value = new_values[0]
                flag = 1
                new_value = new_value.strip()
                break
        if flag == 0:
            for form in forms_3:
                new_values = re.findall(form, text)
                if len(new_values) > 0:
                    new_value = new_values[0].replace(' din ', '/')
                    new_value = new_value.split(" ")[1]
                    new_value = new_value.strip()
                    break
    if ( new_value not in [np.nan]) and ("/" not in new_value):
        flag = 0
        for form in forms_2:
            new_values = re.findall(form, text)
            if len(new_values) > 0:
                flag = 1
                new_value = value + "/" + new_values[0].split(" ")[0]
                new_value = new_value.strip()
                break
    if (flag == 0) and (new_value in [np.nan]):
        for form in forms_4:
            new_values = re.findall(form, text)
            if len(new_values) > 0:
                new_value = new_values[0].split(" ")[1]
                new_value = new_value.strip()
                break
                
    return new_value 

In [89]:
def set_max_floor(current_floor, max_floor, mode_mansarda):
    new_max = max_floor
    if (current_floor == 'mansardă') and (max_floor ==0):
        new_max = mode_mansarda
    elif (current_floor == "parter") or (current_floor == "demisol") or (current_floor == 'mansardă'):
        pass
    elif int(current_floor) > int(max_floor):
        new_max = current_floor
    return new_max   

In [85]:
def dangerous_building(year, max_floor):
    status = None
    year = int(year)
    max_floor = int(max_floor)
    if (year <= 1977) and (max_floor > 5):
        status = "Cade"
    elif (year <= 1977) or (max_floor > 5):
        status = "Te simti norocos?"
    else:
        status = "Sigur"
    return status    

In [65]:
vanzare_df = pd.read_excel("apartamente_de_vanzare.xlsx")
inchiriere_df = pd.read_excel("apartamente_de_inchiriat.xlsx")

In [66]:
vanzare_df.replace("Cere informații", np.nan, inplace=True)
vanzare_df.replace("fără informații", np.nan, inplace=True)

In [67]:
inchiriere_df.replace("Cere informații", np.nan, inplace=True)
inchiriere_df.replace("fără informații", np.nan, inplace=True)

In [68]:
vanzare_df['Pret'] = vanzare_df['Pret'].apply(convert_money)
inchiriere_df['Pret'] = inchiriere_df['Pret'].apply(convert_money)

In [69]:
vanzare_df['Suprafață'] = vanzare_df['Suprafață'].apply(convert_area)
inchiriere_df['Suprafață'] = inchiriere_df['Suprafață'].apply(convert_area)

In [70]:
vanzare_df['Etaj'] = vanzare_df.apply(lambda df: building_floors(df["Etaj"], df["Descriere"]), axis=1)
inchiriere_df['Etaj'] = inchiriere_df.apply(lambda df: building_floors(df["Etaj"], df["Descriere"]), axis=1)

In [71]:
vanzare_df['Etaj apartament'] = vanzare_df['Etaj'].apply(apartment_floor)
inchiriere_df['Etaj apartament'] = inchiriere_df['Etaj'].apply(apartment_floor)

In [72]:
vanzare_df['Numar Etaj Bloc'] = vanzare_df['Etaj'].apply(number_of_floors)
inchiriere_df['Numar Etaj Bloc'] = inchiriere_df['Etaj'].apply(number_of_floors)

In [73]:
vanzare_df["Anul construcției"] = vanzare_df.apply(lambda df: get_year(df["Anul construcției"], df["Descriere"]), axis=1)
inchiriere_df["Anul construcției"] = inchiriere_df.apply(lambda df: get_year(df["Anul construcției"], df["Descriere"]), axis=1)

In [74]:
vanzare_df['Garaj/loc de parcare'] = vanzare_df.apply(lambda df: parcare_or_garaj(df['Garaj/loc de parcare'], df["Descriere"]), axis=1)
inchiriere_df['Garaj/loc de parcare'] = inchiriere_df.apply(lambda df: parcare_or_garaj(df['Garaj/loc de parcare'], df["Descriere"]), axis=1)

In [75]:
new_vanzare_df = vanzare_df.loc[:,("Zona", "Pret", 'Suprafață', 'Numărul de camere', 'Anul construcției',
                                   'Etaj apartament', 'Numar Etaj Bloc', 'Garaj/loc de parcare')]
new_inchiriere_df = inchiriere_df.loc[:, ("Zona", "Pret", 'Suprafață', 'Numărul de camere', 'Anul construcției',
                                          'Etaj apartament', 'Numar Etaj Bloc', 'Garaj/loc de parcare')]

In [76]:
new_vanzare_df['Anul construcției'].replace(np.nan, 0, inplace=True)
new_vanzare_df['Anul construcției'] = new_vanzare_df['Anul construcției'].astype('int')
new_inchiriere_df["Anul construcției"].replace(np.nan, 0, inplace=True)
new_inchiriere_df["Anul construcției"] = new_inchiriere_df["Anul construcției"].astype("int")

In [77]:
new_vanzare_df['Numar Etaj Bloc'].replace(np.nan, 0, inplace=True)
index_to_drop = new_vanzare_df.index[new_vanzare_df['Numar Etaj Bloc'] == "24"].to_list()
new_vanzare_df.drop(index_to_drop, inplace=True)

In [78]:
new_inchiriere_df['Numar Etaj Bloc'].replace(np.nan, 0, inplace=True)
index_to_drop = new_inchiriere_df.index[new_inchiriere_df['Numar Etaj Bloc'] == "24"].to_list()
new_inchiriere_df.drop(index_to_drop, inplace=True)

In [79]:
new_vanzare_df['Garaj/loc de parcare'].replace(np.nan, "Fara loc", inplace=True)
new_inchiriere_df['Garaj/loc de parcare'].replace(np.nan, "Fara loc", inplace=True)

In [80]:
new_vanzare_df['Etaj apartament'].replace(np.nan, 0, inplace=True)
index_to_drop = new_vanzare_df[new_vanzare_df['Etaj apartament'] == 0].index
new_vanzare_df.drop(index_to_drop, inplace=True)

In [81]:
new_inchiriere_df['Etaj apartament'].replace(np.nan, 0, inplace=True)
index_to_drop = new_inchiriere_df[new_inchiriere_df['Etaj apartament'] == 0].index
new_inchiriere_df.drop(index_to_drop, inplace=True)

In [92]:
mode_mansarda = int(new_vanzare_df[new_vanzare_df['Etaj apartament'] == 'mansardă']['Numar Etaj Bloc'].mode().to_list()[0])
new_vanzare_df['Numar Etaj Bloc'] = new_vanzare_df.apply(lambda df:
                                                         set_max_floor(df['Etaj apartament'], df['Numar Etaj Bloc'], mode_mansarda), axis=1)

In [93]:
mode_mansarda = int(new_inchiriere_df[new_inchiriere_df['Etaj apartament'] == 'mansardă']['Numar Etaj Bloc'].mode().to_list()[0])
new_inchiriere_df['Numar Etaj Bloc'] = new_inchiriere_df.apply(lambda df:
                                                         set_max_floor(df['Etaj apartament'], df['Numar Etaj Bloc'], mode_mansarda), axis=1)

In [96]:
new_vanzare_df['Cutremur'] = new_vanzare_df.apply(lambda df:
                                                  dangerous_building(df['Anul construcției'], df['Numar Etaj Bloc']), axis=1)
new_inchiriere_df['Cutremur'] = new_inchiriere_df.apply(lambda df:
                                                  dangerous_building(df['Anul construcției'], df['Numar Etaj Bloc']), axis=1)
